In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def cargar_imagen(ruta_imagen):
    """
    Carga una imagen desde la ruta dada en BGR y la convierte a RGB para 
    visualizar con Matplotlib, además de la escala de grises.
    Retorna:
        - img_rgb: Imagen en formato RGB
        - img_gray: Imagen original en escala de grises
    """
    img_bgr = cv2.imread(ruta_imagen)
    if img_bgr is None:
        raise ValueError(f"No se pudo cargar la imagen en la ruta: {ruta_imagen}")

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    return img_rgb, img_gray

In [3]:
def mostrar_imagen(img_rgb, titulo="Imagen"):
    """
    Muestra una imagen en RGB con Matplotlib.
    """
    plt.imshow(img_rgb)
    plt.title(titulo)
    plt.axis("off")
    plt.show()

In [4]:
def filtro_paso_alto_gaussiano(img_gray, ksize=5):
    """
    Aplica un filtro de paso alto a la imagen en escala de grises (img_gray)
    restando la versión suavizada (Gaussiana) de la imagen original.
    Retorna la imagen realzada en altas frecuencias.

    Parámetros:
    - ksize: Tamaño del kernel del filtro Gaussiano (debe ser impar).
    """
    # Paso bajo usando filtro Gaussiano
    suavizada = cv2.GaussianBlur(img_gray, (ksize, ksize), 0)
    # Paso alto = imagen original - imagen suavizada
    altas_frec = cv2.subtract(img_gray, suavizada)
    return altas_frec

In [5]:
def medir_alta_frecuencia(img_gray, ksize=5, funcion_filtro=filtro_paso_alto_gaussiano):
    """
    Calcula una medida de 'energía' (varianza) de la imagen de alta frecuencia.
    Retorna la varianza resultante.
    """
    hf = funcion_filtro(img_gray, ksize=ksize)
    return np.var(hf)

In [6]:
def procesar_imagenes_carpeta_gaussiano(
    carpeta_entrada="images",
    carpeta_salida="images_alta_frecuencia",
    ksize=5
):
    """
    Recorre todas las subcarpetas dentro de 'carpeta_entrada'.
    - Para cada archivo .png, aplica el filtro de paso alto Gaussiano.
    - Guarda la imagen resultante en 'carpeta_salida', manteniendo la misma estructura.
    - Genera un archivo .txt con la varianza de cada imagen procesada.

    Parámetros:
    - carpeta_entrada: ruta de la carpeta de entrada.
    - carpeta_salida: ruta de la carpeta donde se guardarán los resultados.
    - ksize: Tamaño del kernel para el filtro Gaussiano.
    """
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'gaussiano'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "gaussiano")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar las varianzas
        ruta_txt = os.path.join(carpeta_salida_sub, "info_varianzas_gaussiano.txt")
        
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Varianzas de alta frecuencia (filtro gaussiano)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("La varianza indica la dispersión de los valores de alta frecuencia.\n")
            archivo_txt.write("Valores mayores suelen corresponder a más bordes y mayor nitidez.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(".png"):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en escala de grises
                    img_bgr = cv2.imread(ruta_imagen_entrada)
                    if img_bgr is None:
                        continue
                    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
                    
                    # Filtro de paso alto Gaussiano
                    altas_frec_img = filtro_paso_alto_gaussiano(img_gray, ksize=ksize)
                    
                    # Calcular varianza
                    varianza = medir_alta_frecuencia(img_gray, ksize=ksize, funcion_filtro=filtro_paso_alto_gaussiano)
                    
                    # Escalado para guardar la imagen con mayor contraste
                    hf_min, hf_max = altas_frec_img.min(), altas_frec_img.max()
                    if hf_max > hf_min:
                        altas_frec_escalada = (altas_frec_img - hf_min) / (hf_max - hf_min) * 255.0
                    else:
                        altas_frec_escalada = altas_frec_img.astype(np.float32)
                        
                    altas_frec_escalada = altas_frec_escalada.astype(np.uint8)
                    
                    # Construir ruta de salida
                    nombre, extension = os.path.splitext(filename)
                    nombre_salida = f"{nombre}_alta_frecuencia_gaussiano.png"
                    ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)
                    
                    # Guardar imagen
                    cv2.imwrite(ruta_imagen_salida, altas_frec_escalada)
                    
                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> varianza: {varianza:.3f}\n")

    print("Procesamiento completado con filtro Gaussiano.")

In [7]:
# Ejecuta el procesamiento
procesar_imagenes_carpeta_gaussiano(
    carpeta_entrada="images",
    carpeta_salida="images_high_frec",
    ksize=5
)

Procesamiento completado con filtro Gaussiano.
